In [173]:
import boxes

# Read file

In [174]:
filename = "13-2014-0830-6260-LA93-0M20-E080.jp2"
data = boxes.read_jp2(filename)

# Read boxes

In [175]:
boxes_bytes = read_boxes(data)

In [176]:
boxes = parse_boxes_bytes(boxes_bytes)

In [177]:
for box in boxes:
    print(box)

A quiet signature box
A type box claiming compatibility with b'jp2 '
A HeaderBox containing :
    A ImageHeaderBox filled with the data :
        NC_nb_components: 3
        IPR_intellectual_property: 0
        BPC_bits_per_components: 7
        C_compression_type: 7
        width: 25000
        height: 25000
        UnkC_colourspace_unknown: 1
    A ColourSpecificationBox filled with the data :
        METH_specification_method: 1
        EnumCS_enumerated_colourspace: 16
        APPROX_colourspace_approximation: 0
        PREC_precedence: 0
    A ResolutionBox containing :
        A DefaultDisplayResolutionBox filled with the data :
            VRdE: 2
            HRdN: 12901
            HRdE: 2
            VRdD: 32768
            HRdD: 32768
            VRdN: 12901
A UUID box with ID b'\xb1K\xf8\xbd\x08=KC\xa5\xae\x8c\xd7\xd5\xa6\xce\x03' and DATA b'II*\x00\t\x00\x00\x00\x00\x11\x00\x00\x01\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x01\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x